# Neo4j Lab 11: Python Driver & Service Architecture
## Part 2: Pydantic Data Models & Type Safety

**Duration:** 10 minutes  
**Objective:** Implement type-safe data models with Pydantic for insurance entities with comprehensive validation

---

## Overview

This notebook covers:
- Pydantic model creation for insurance entities
- Data validation and type safety
- Custom validators for business rules
- Enum types for controlled values
- Model inheritance and composition

## Cell 1: Import Required Dependencies

First, ensure we have the connection manager from the previous notebook and import Pydantic.

In [ ]:
# Cell 1: Import dependencies
# Note: Make sure you've run notebook 01 first to initialize connection_manager

# If connection_manager is not available, uncomment and run:
# %run 01_python_driver_setup_and_basics.ipynb

from pydantic import BaseModel, Field, validator, EmailStr
from typing import Optional, List, Dict, Any, Literal
from datetime import datetime, date
from enum import Enum
import uuid

print("✓ Dependencies imported successfully")

## Cell 2: Define Enumerations for Controlled Values

Create enum types for policy status, policy types, claim status, and risk levels.

In [ ]:
# Cell 2: Enumerations for controlled values
print("📊 CREATING PYDANTIC DATA MODELS:")
print("=" * 50)

# Enums for controlled values
class PolicyStatus(str, Enum):
    ACTIVE = "Active"
    INACTIVE = "Inactive"
    CANCELLED = "Cancelled"
    SUSPENDED = "Suspended"

class PolicyType(str, Enum):
    AUTO = "Auto"
    HOME = "Homeowner"
    LIFE = "Life"
    HEALTH = "Health"
    COMMERCIAL = "Commercial"

class ClaimStatus(str, Enum):
    FILED = "Filed"
    INVESTIGATING = "Investigating"
    APPROVED = "Approved"
    DENIED = "Denied"
    SETTLED = "Settled"

class RiskLevel(str, Enum):
    LOW = "Low Risk"
    MEDIUM = "Medium Risk"
    HIGH = "High Risk"
    VERY_HIGH = "Very High Risk"

print("✓ Enumerations defined:")
print(f"  - PolicyStatus: {[s.value for s in PolicyStatus]}")
print(f"  - PolicyType: {[t.value for t in PolicyType]}")
print(f"  - ClaimStatus: {[c.value for c in ClaimStatus]}")
print(f"  - RiskLevel: {[r.value for r in RiskLevel]}")
print("=" * 50)

## Cell 3: Base Entity Model

Create a base model with common fields for all entities.

In [ ]:
# Cell 3: Base entity model with common fields

class BaseEntity(BaseModel):
    """Base model with common entity fields"""
    id: Optional[str] = Field(default_factory=lambda: str(uuid.uuid4()))
    created_at: Optional[datetime] = Field(default_factory=datetime.now)
    updated_at: Optional[datetime] = Field(default_factory=datetime.now)
    version: Optional[int] = Field(default=1)
    
    class Config:
        use_enum_values = True
        json_encoders = {
            datetime: lambda v: v.isoformat(),
            date: lambda v: v.isoformat()
        }

print("✓ Base entity model created with:")
print("  - Automatic ID generation")
print("  - Timestamp tracking (created_at, updated_at)")
print("  - Version control for optimistic locking")
print("  - JSON serialization support")

## Cell 4: Customer Data Models

Define customer models with validation rules for business requirements.

In [ ]:
# Cell 4: Customer data models with validation

class CustomerBase(BaseModel):
    """Base customer model"""
    customer_id: str = Field(..., description="Unique customer identifier")
    first_name: str = Field(..., min_length=1, max_length=50)
    last_name: str = Field(..., min_length=1, max_length=50)
    email: EmailStr
    phone: Optional[str] = Field(None, regex=r'^\+?1?\d{9,15}$')
    date_of_birth: date
    
    @validator('customer_id')
    def customer_id_format(cls, v):
        if not v.startswith('CUST-'):
            raise ValueError('Customer ID must start with CUST-')
        return v
    
    @validator('date_of_birth')
    def validate_age(cls, v):
        today = date.today()
        age = today.year - v.year - ((today.month, today.day) < (v.month, v.day))
        if age < 18 or age > 120:
            raise ValueError('Customer must be between 18 and 120 years old')
        return v

class CustomerCreate(CustomerBase):
    """Model for creating new customers"""
    initial_contact_method: Optional[str] = "Web"
    referral_source: Optional[str] = None

class Customer(BaseEntity, CustomerBase):
    """Complete customer model"""
    customer_since: Optional[date] = None
    total_policies: Optional[int] = Field(default=0, ge=0)
    total_claims: Optional[int] = Field(default=0, ge=0)
    customer_value: Optional[float] = Field(default=0.0, ge=0)
    risk_score: Optional[float] = Field(default=0.0, ge=0, le=100)

print("✓ Customer models created:")
print("  - CustomerBase: Base customer data with validation")
print("  - CustomerCreate: Model for new customer creation")
print("  - Customer: Complete customer with metrics")
print("\n✓ Validation rules:")
print("  - Customer ID must start with 'CUST-'")
print("  - Age must be between 18 and 120")
print("  - Email must be valid format")
print("  - Phone must match international format")

## Cell 5: Policy Data Models

Create policy models with business logic validation.

In [ ]:
# Cell 5: Policy data models with validation

class PolicyBase(BaseModel):
    """Base policy model"""
    policy_number: str = Field(..., description="Unique policy identifier")
    policy_type: PolicyType
    customer_id: str
    effective_date: date
    expiration_date: date
    premium_amount: float = Field(..., gt=0, description="Monthly premium amount")
    coverage_amount: float = Field(..., gt=0, description="Total coverage amount")
    deductible: Optional[float] = Field(default=0, ge=0)
    
    @validator('policy_number')
    def policy_number_format(cls, v):
        if not (v.startswith('POL-') and len(v) >= 8):
            raise ValueError('Policy number must start with POL- and be at least 8 characters')
        return v
    
    @validator('expiration_date')
    def expiration_after_effective(cls, v, values):
        if 'effective_date' in values and v <= values['effective_date']:
            raise ValueError('Expiration date must be after effective date')
        return v

class PolicyCreate(PolicyBase):
    """Model for creating new policies"""
    agent_id: Optional[str] = None
    underwriting_notes: Optional[str] = None

class Policy(BaseEntity, PolicyBase):
    """Complete policy model"""
    policy_status: PolicyStatus = PolicyStatus.ACTIVE
    last_payment_date: Optional[date] = None
    next_payment_due: Optional[date] = None
    claims_count: Optional[int] = Field(default=0, ge=0)
    total_claims_amount: Optional[float] = Field(default=0.0, ge=0)

print("✓ Policy models created:")
print("  - PolicyBase: Base policy data with validation")
print("  - PolicyCreate: Model for new policy creation")
print("  - Policy: Complete policy with status tracking")
print("\n✓ Validation rules:")
print("  - Policy number must start with 'POL-'")
print("  - Expiration date must be after effective date")
print("  - Premium and coverage amounts must be positive")
print("  - Policy type must be from defined enum")

## Cell 6: Claim Data Models

Define claim models with incident date validation.

In [ ]:
# Cell 6: Claim data models with validation

class ClaimBase(BaseModel):
    """Base claim model"""
    claim_number: str = Field(..., description="Unique claim identifier")
    policy_number: str
    claim_date: date
    incident_date: date
    claim_amount: float = Field(..., gt=0)
    description: str = Field(..., min_length=10, max_length=1000)
    
    @validator('claim_number')
    def claim_number_format(cls, v):
        if not (v.startswith('CLM-') and len(v) >= 8):
            raise ValueError('Claim number must start with CLM- and be at least 8 characters')
        return v
    
    @validator('incident_date')
    def incident_before_claim(cls, v, values):
        if 'claim_date' in values and v > values['claim_date']:
            raise ValueError('Incident date cannot be after claim date')
        return v

class ClaimCreate(ClaimBase):
    """Model for creating new claims"""
    adjuster_id: Optional[str] = None
    priority: Optional[Literal["Low", "Medium", "High", "Critical"]] = "Medium"

class Claim(BaseEntity, ClaimBase):
    """Complete claim model"""
    claim_status: ClaimStatus = ClaimStatus.FILED
    adjuster_id: Optional[str] = None
    settlement_amount: Optional[float] = Field(default=0.0, ge=0)
    settlement_date: Optional[date] = None
    investigation_notes: Optional[str] = None

print("✓ Claim models created:")
print("  - ClaimBase: Base claim data with validation")
print("  - ClaimCreate: Model for new claim creation")
print("  - Claim: Complete claim with status tracking")
print("\n✓ Validation rules:")
print("  - Claim number must start with 'CLM-'")
print("  - Incident date must be before or equal to claim date")
print("  - Claim amount must be positive")
print("  - Description must be 10-1000 characters")

## Cell 7: Risk Assessment Model

Create a risk assessment model for tracking customer risk profiles.

In [ ]:
# Cell 7: Risk assessment model

class RiskAssessment(BaseEntity):
    """Risk assessment model"""
    customer_id: str
    assessment_date: date
    risk_level: RiskLevel
    risk_score: float = Field(..., ge=0, le=100)
    factors: List[str] = Field(default_factory=list)
    recommendations: List[str] = Field(default_factory=list)
    next_review_date: Optional[date] = None

print("✓ Risk assessment model created:")
print("  - Tracks customer risk profiles over time")
print("  - Risk score range: 0-100")
print("  - Risk levels: Low, Medium, High, Very High")
print("  - Supports risk factors and recommendations")

## Cell 8: Test Data Model Validation

Validate all models with test data to ensure validation rules work correctly.

In [ ]:
# Cell 8: Test data model validation
print("🧪 TESTING DATA MODEL VALIDATION:")
print("=" * 50)

try:
    # Test 1: Valid customer
    print("\nTest 1: Valid Customer Creation")
    customer_data = {
        "customer_id": "CUST-123456",
        "first_name": "John",
        "last_name": "Smith",
        "email": "john.smith@email.com",
        "phone": "+1234567890",
        "date_of_birth": "1985-03-15"
    }
    customer = Customer(**customer_data)
    print("✓ Customer model validation passed")
    print(f"  Customer: {customer.first_name} {customer.last_name}")
    print(f"  Email: {customer.email}")
    print(f"  Age: {(date.today() - customer.date_of_birth).days // 365} years")
    
    # Test 2: Valid policy
    print("\nTest 2: Valid Policy Creation")
    policy_data = {
        "policy_number": "POL-AUTO-001",
        "policy_type": "Auto",
        "customer_id": "CUST-123456",
        "effective_date": "2025-01-01",
        "expiration_date": "2026-01-01",
        "premium_amount": 150.00,
        "coverage_amount": 25000.00,
        "deductible": 500.00
    }
    policy = Policy(**policy_data)
    print("✓ Policy model validation passed")
    print(f"  Policy: {policy.policy_number}")
    print(f"  Type: {policy.policy_type}")
    print(f"  Premium: ${policy.premium_amount}/month")
    print(f"  Coverage: ${policy.coverage_amount:,.2f}")
    
    # Test 3: Valid claim
    print("\nTest 3: Valid Claim Creation")
    claim_data = {
        "claim_number": "CLM-2025-001",
        "policy_number": "POL-AUTO-001",
        "claim_date": "2025-07-18",
        "incident_date": "2025-07-17",
        "claim_amount": 2500.00,
        "description": "Minor fender bender in parking lot with damage to rear bumper"
    }
    claim = Claim(**claim_data)
    print("✓ Claim model validation passed")
    print(f"  Claim: {claim.claim_number}")
    print(f"  Amount: ${claim.claim_amount:,.2f}")
    print(f"  Status: {claim.claim_status}")
    
    # Test 4: Invalid customer ID (should fail)
    print("\nTest 4: Invalid Customer ID (Expected to Fail)")
    try:
        invalid_customer = CustomerCreate(
            customer_id="INVALID-123",  # Wrong prefix
            first_name="Test",
            last_name="User",
            email="test@example.com",
            date_of_birth="1990-01-01"
        )
        print("✗ Validation should have failed!")
    except ValueError as e:
        print(f"✓ Validation correctly rejected: {e}")
    
    # Test 5: Invalid age (should fail)
    print("\nTest 5: Invalid Age (Expected to Fail)")
    try:
        young_customer = CustomerCreate(
            customer_id="CUST-YOUNG",
            first_name="Too",
            last_name="Young",
            email="young@example.com",
            date_of_birth="2010-01-01"  # Too young (under 18)
        )
        print("✗ Validation should have failed!")
    except ValueError as e:
        print(f"✓ Validation correctly rejected: {e}")
    
    # Test 6: Invalid policy dates (should fail)
    print("\nTest 6: Invalid Policy Dates (Expected to Fail)")
    try:
        invalid_policy = PolicyCreate(
            policy_number="POL-TEST-001",
            policy_type=PolicyType.AUTO,
            customer_id="CUST-123456",
            effective_date=date(2025, 12, 31),
            expiration_date=date(2025, 1, 1),  # Before effective date
            premium_amount=100.00,
            coverage_amount=20000.00
        )
        print("✗ Validation should have failed!")
    except ValueError as e:
        print(f"✓ Validation correctly rejected: {e}")
    
    print("\n" + "=" * 50)
    print("✓ All data model validation tests completed successfully!")
    print("=" * 50)
    
except Exception as e:
    print(f"\n✗ Model validation testing failed: {e}")
    import traceback
    traceback.print_exc()

## Summary

In this notebook, you've:

1. ✅ Created enumeration types for controlled values (PolicyStatus, PolicyType, ClaimStatus, RiskLevel)
2. ✅ Implemented a base entity model with common fields and configuration
3. ✅ Built comprehensive customer models with validation rules:
   - Customer ID format validation
   - Age range validation (18-120)
   - Email format validation
   - Phone number format validation
4. ✅ Created policy models with business logic validation:
   - Policy number format
   - Date range validation
   - Positive amount validation
5. ✅ Developed claim models with incident date validation
6. ✅ Implemented risk assessment model for tracking customer risk
7. ✅ Tested all validation rules with positive and negative test cases

**Key Benefits:**
- Type safety at runtime
- Automatic data validation
- Self-documenting code
- JSON serialization support
- IDE autocomplete and type hints

**Next Steps:** Proceed to `03_repository_and_service_layer.ipynb` to implement the repository pattern and service layer.